In [1]:
from google.colab import drive
import sys
import os
import random
import numpy as np
import cv2

# Mount Google Drive
drive.mount('/content/drive/')

wd = '/content/drive/My Drive/Colab Notebooks/Capstone/'

Mounted at /content/drive/


In [2]:
wdfiles = '/content/drive/My Drive/Colab Notebooks/Capstone/Upload/'

!mkdir train_local
!cp -r '/content/drive/My Drive/Colab Notebooks/Capstone/Upload/' -d train_local

In [3]:
!nvidia-smi

Tue Apr 12 18:46:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install datasets
!pip install transformers

     |████████████████████████████████| 325 kB 13.8 MB/s 
     |████████████████████████████████| 136 kB 71.0 MB/s 
     |████████████████████████████████| 77 kB 7.7 MB/s 
     |████████████████████████████████| 1.1 MB 62.9 MB/s 
     |████████████████████████████████| 212 kB 74.2 MB/s 
     |████████████████████████████████| 127 kB 75.7 MB/s 
     |████████████████████████████████| 144 kB 70.1 MB/s 
     |████████████████████████████████| 271 kB 77.2 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 4.0 MB 13.8 MB/s

In [5]:

#Non-Specific Imports
import os
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
import csv




#NLP Imports
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer


#Modeling Imports
import torch
from datasets import load_dataset
from datasets import load_metric
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import get_scheduler
from torch.utils.data import DataLoader
from torch.optim import AdamW

In [6]:
################################
#
#  Modeling with ELECTRA
#
################################


#Load Electra - Model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/electra-base-discriminator")
model = AutoModelForSequenceClassification.from_pretrained("google/electra-base-discriminator")



Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.o

In [7]:
#Load Data
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': wdfiles+'New_Samples_pandemic_reordered_3000_train.csv',
                                          'validation':wdfiles+'New_Samples_pandemic_reordered_3000_validation.csv'
                                          })


Using custom data configuration default-08ef68b4922ad42d


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-08ef68b4922ad42d/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
###  Set Seeds  ###
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


###  Tokenize  ###
def tokenize_function(example):
    return tokenizer(example["Review_Body"],truncation=True)


#Prepare the data
tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#Remove raw text - already tokenized and masked
tokenized_datasets = tokenized_datasets.remove_columns(["Review_Body"])

#Note: Removing all non-text columns?
tokenized_datasets = tokenized_datasets.remove_columns(['Unnamed: 0'])

#Rename and reformat columns
tokenized_datasets = tokenized_datasets.rename_column("Review_rating", "labels")
tokenized_datasets.set_format("torch", columns=tokenized_datasets["train"].column_names)
tokenized_datasets["train"].column_names


###  Dataloader  ###
train_dataloader = DataLoader(tokenized_datasets["train"],
                              shuffle=True, batch_size=12, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_datasets["validation"],
                             batch_size=12, collate_fn=data_collator)


for batch in train_dataloader:
    break

  0%|          | 0/88 [00:00<?, ?ba/s]

  0%|          | 0/30 [00:00<?, ?ba/s]

In [9]:
device = torch.device("cuda")
device

device(type='cuda')

In [10]:

###  Set-up  ###
num_epochs = 3
checkpoint = "google/electra-base-discriminator"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)



###  Model Set-up  ###
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=5)
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)
optimizer = AdamW(model.parameters(), lr=5e-5)# Normally e-5

num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler( "linear", optimizer=optimizer,
                              num_warmup_steps=0, num_training_steps=num_training_steps)
print(num_training_steps)


model.to(device)

#Tqdm
progress_bar = tqdm(range(num_training_steps))


model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    #Save at the end of each epoch
    model.save_pretrained(wd+'Electra_epoch_'+str(epoch)+'_reordered_pandemic_features_model_save')


pred_test=[]
###  Evaluations  ###
metric = load_metric("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])
    pred_test.append(predictions.cpu().numpy())

metric.compute()

with open(wd+'Electra_reordered_val_pandemic_predictions.csv', 'w') as f: 
    write = csv.writer(f) 
    write.writerows(pred_test) 

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.o

tensor(1.5704, grad_fn=<NllLossBackward0>) torch.Size([12, 5])
21966


100%|██████████| 21966/21966 [1:45:17<00:00,  3.92it/s]

Electra if you always guess 4 : Accuracy 0.4863  
#--------------------------------   
Electra on samples, 3 epoch : Accuracy 0.6850 
Electra on samples with added features, 3 epoch : Accuracy 0.76  
Electra on samples with added features, 5 epoch : Accuracy 0.745


Accuracy on val with 2000 per state: 0.733529
Accuracy on val with 5000 per state: 0.4921 ???

In [13]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

def tidy_predictions(pred_in, col_name=''):

  #Clean the predictions
  pred_melt = pd.DataFrame(pred_in).T.melt()
  pred_melt = pred_melt.drop('variable', axis=1)
  pred_melt = pred_melt.rename({'value':'Prediction'+col_name}, axis=1)
  pred_melt['Prediction'+col_name] = pred_melt['Prediction'+col_name]#+1
  pred_melt = pred_melt.dropna(axis=0)

  return pred_melt



#Load the test labels
labels_test = pd.read_csv(wd+'New_Samples_pandemic_reordered_3000_validation.csv')
labels_test=labels_test["Review_rating"]

pred_tidy = tidy_predictions(pred_test, col_name ='_Baseline')

mse = mean_squared_error(labels_test, pred_tidy)
acc = accuracy_score(labels_test, pred_tidy)

print(f'Acc:{acc}\nMSE:{mse}')


Acc:0.7720568150778476
MSE:0.2778612400983338


In [14]:

pred_tidy.tail(30)

,Prediction_Baseline
29258,1.0
29259,4.0
29260,4.0
29261,4.0
29262,4.0
29263,4.0
29264,4.0
29265,3.0
29266,4.0
29267,3.0
